## Import libraries

In [ ]:
import celloracle as co
from celloracle.applications import Pseudotime_calculator
from celloracle.applications import Gradient_calculator
from celloracle.applications import Oracle_development_module
import anndata2ri
import os
import sys
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import copy
import glob
import time
import shutil
from tqdm.auto import tqdm

## Plotting parameters settings

In [ ]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [ ]:
save_folder = "Scarfo_HEC2023/scRNAseq"
os.makedirs(save_folder, exist_ok=True)

## Load Seurat object in Python

In [ ]:
# Activate the anndata2ri conversion between SingleCellExperiment and AnnData
anndata2ri.activate()
#Loading the rpy2 extension
%load_ext rpy2.ipython
sc.settings.verbosity = 3
sc.logging.print_versions()

In [ ]:
%%R
suppressPackageStartupMessages(library(Seurat))
runx1_obj <- readRDS("Scarfo_HEC2023/scRNAseq/RUNX1_clusters.rds")

In [ ]:
%%R -o runx1_obj_sce
#convert the Seurat object to a SingleCellExperiment object
runx1_obj_sce <- as.SingleCellExperiment(runx1_obj)
runx1_obj_sce

In [ ]:
adata = runx1_obj_sce
adata

In [ ]:
print(f"Cell number is :{adata.shape[0]}")
print(f"Gene number is :{adata.shape[1]}")

In [ ]:
sc.pl.umap(adata, color='RNA_snn_new_res.0.6')

## Constructing GRN

In [ ]:
# Select top 3000 highly-variable genes
filter_result = sc.pp.filter_genes_dispersion(adata.X,flavor='cell_ranger',n_top_genes=3000,log=False)
# Subset the genes
adata = adata[:, filter_result.gene_subset]
print(f"Cell number is :{adata.shape[0]}")
print(f"Gene number is :{adata.shape[1]}")

In [ ]:
# Load data from CellTalkDB
ligand_receptor = pd.read_csv("Scarfo_HEC2023/scRNAseq/CellTalkDB_modified.txt", sep='\t')
ligand_receptor.columns = ['Ligand','Receptor']
ligand_receptor
print('FCGR2B' in ligand_receptor['Ligand'].unique())

In [ ]:
# Make dictionary: dictionary key is Ligand and dictionary value is list of target genes.
LR_to_TG_dictionary = {}
for LR, TGs in zip(ligand_receptor.Ligand, ligand_receptor.Receptor):
    TG_list = TGs.replace(" ", "").split(",")
    LR_to_TG_dictionary[LR] = TG_list
TG_to_LR_dictionary = co.utility.inverse_dictionary(LR_to_TG_dictionary)
TG_to_LR_dictionary

In [ ]:
# Instantiate Oracle object
oracle = co.Oracle()
oracle.import_anndata_as_raw_count(adata=adata,cluster_column_name="RNA_snn_new_res.0.6",embedding_name="X_umap")

In [ ]:
# Add LR information 
oracle.addTFinfo_dictionary(TG_to_LR_dictionary)

In [ ]:
# Perform PCA
oracle.perform_PCA()
# Select important PCs
plt.plot(np.cumsum(oracle.pca.explained_variance_ratio_)[:100])
n_comps = np.where(np.diff(np.diff(np.cumsum(oracle.pca.explained_variance_ratio_))>0.002))[0][0]
plt.axvline(n_comps, c="k")
plt.show()
print(n_comps)
n_comps = min(n_comps, 50)

In [ ]:
# Estimate the optimal number of nearest neighbors for KNN imputation
n_cell = oracle.adata.shape[0]
print(f"cell number is :{n_cell}")

In [ ]:
k = int(0.025*n_cell)
print(f"Auto-selected k is :{k}")

In [ ]:
# perform KNN imputation
oracle.knn_imputation(n_pca_dims=n_comps,k=k, balanced=True,b_sight=k*8,b_maxl=k*4,n_jobs=4)

In [ ]:
# GRN calculation
%time
links = oracle.get_links(cluster_name_for_GRN_unit="RNA_snn_new_res.0.6", alpha=10,verbose_level=10)

In [ ]:
links.links_dict.keys()

In [ ]:
# filter weak edges
links.filter_links(p=0.001, weight="coef_abs", threshold_number=2000)

In [ ]:
# network degree distribution
links.plot_degree_distributions(plot_model=True,
                                save=f"{save_folder}/degree_distribution/",)

In [ ]:
# Calculate network scores.
links.get_network_score()

## Adding pseudotime to oracle object

In [ ]:
# Instantiate pseudotime object using oracle object.
pt = Pseudotime_calculator(oracle_object=oracle)

In [ ]:
clusters_in_RUNX1_lineage = ['0','1','2','11','16','17']
# Make a dictionary
lineage_dictionary = {"Lineage_RUNX1": clusters_in_RUNX1_lineage}
# Input lineage information into pseudotime object
pt.set_lineage(lineage_dictionary=lineage_dictionary)
# Visualize lineage information
pt.plot_lineages()

In [ ]:
# add root cell information
# plotly is required
try:
    import plotly.express as px
    def plot(adata, embedding_key, cluster_column_name):
        embedding = adata.obsm[embedding_key]
        df = pd.DataFrame(embedding, columns=["x", "y"])
        df["cluster"] = adata.obs[cluster_column_name].values
        df["label"] = adata.obs.index.values
        fig = px.scatter(df, x="x", y="y", hover_name=df["label"], color="cluster")
        fig.show()
    plot(adata=pt.adata,
         embedding_key=pt.obsm_key,
         cluster_column_name=pt.cluster_column_name)
except:
    print("Plotly not found in your environment. Did you install plotly?")

In [ ]:
# Estimated root cell name for each lineage
root_cells = {"Lineage_RUNX1": "Sample1_GGAGAACTCGCGGTAC-1"}
pt.set_root_cells(root_cells=root_cells)

In [ ]:
# Check diffusion map data.
"X_diffmap" in pt.adata.obsm

In [ ]:
# calculate diffusion map
sc.pp.neighbors(pt.adata, n_neighbors=30)
sc.tl.diffmap(pt.adata)

In [ ]:
# Calculate pseudotime
pt.get_pseudotime_per_each_lineage()
# Check results
pt.plot_pseudotime(cmap="rainbow")

In [ ]:
# Add calculated pseudotime data to the oracle object
oracle.adata.obs = pt.adata.obs

## In silico perturbation analysis

In [ ]:
# make pridictive models for simulation and fit the ridge regression models again
links.filter_links()
oracle.get_cluster_specific_TFdict_from_Links(links_object=links)
oracle.fit_GRN_for_simulation(alpha=10, use_cluster_specific_TFdict=True)

In [ ]:
goi = "FCGR2B"
# Enter perturbation conditions to simulate signal propagation after the perturbation.
oracle.simulate_shift(perturb_condition={goi: 0.0}, n_propagation=3)

In [ ]:
# Get transition probability
oracle.estimate_transition_prob(n_neighbors=200,knn_random=True,sampled_fraction=1)
# Calculate embedding
oracle.calculate_embedding_shift(sigma_corr=0.05)

In [ ]:
# quiver plots
fig, ax = plt.subplots(1, 2,  figsize=[13, 6])
scale = 25
# Show quiver plot
oracle.plot_quiver(scale=scale, ax=ax[0])
ax[0].set_title(f"Simulated cell identity shift vector: {goi} KO")
# Show quiver plot that was calculated with randomized graph.
oracle.plot_quiver_random(scale=scale, ax=ax[1])
ax[1].set_title(f"Randomized simulation vector")
plt.show()

In [ ]:
# establishing digitalized grids
n_grid = 40
oracle.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)

In [ ]:
# Search for best min_mass.
oracle.suggest_mass_thresholds(n_suggestion=12)

In [ ]:
min_mass = 19
oracle.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
# shift vector plots
fig, ax = plt.subplots(1, 2,  figsize=[13, 5])
scale_simulation = 2
# Show quiver plot
oracle.plot_simulation_flow_on_grid(scale=scale_simulation, ax=ax[0])
ax[0].set_title(f"Simulated cell identity shift vector: {goi} KO")
# Show quiver plot that was calculated with randomized graph.
oracle.plot_simulation_flow_random_on_grid(scale=scale_simulation, ax=ax[1])
ax[1].set_title(f"Randomized simulation vector")
plt.show()

In [ ]:
# Plot vector field with cell cluster
fig, ax = plt.subplots(figsize=[8, 8])
oracle.plot_cluster_whole(ax=ax, s=10)
oracle.plot_simulation_flow_on_grid(scale=scale_simulation, ax=ax, show_background=False)

In [ ]:
# Visualize pseudotime
fig, ax = plt.subplots(figsize=[6,6])
sc.pl.embedding(adata=oracle.adata, basis=oracle.embedding_name, ax=ax, cmap="viridis",
                color=["Pseudotime"], save="pseudotime.pdf")

In [ ]:
# Instantiate Gradient calculator object
gradient = Gradient_calculator(oracle_object=oracle, pseudotime_key="Pseudotime")

In [ ]:
gradient.calculate_p_mass(smooth=0.8, n_grid=n_grid, n_neighbors=200)
gradient.calculate_mass_filter(min_mass=min_mass, plot=True)

In [ ]:
# convert pseudotime intop grid points
gradient.transfer_data_into_grid(args={"method": "polynomial", "n_poly":5}, plot=True)

In [ ]:
# Calculate graddient
gradient.calculate_gradient()
# Show results
scale_dev = 20
gradient.visualize_results(scale=scale_dev, s=5)

In [ ]:
# Visualize results
fig, ax = plt.subplots(figsize=[6, 6])
gradient.plot_dev_flow_on_grid(scale=scale_dev, ax=ax)
ax.set_title(f"Normal Developmental flow")

In [ ]:
# quantitatively compare the directionality and size of vectors between perturbation simulation and natural differentiation using inner product
# Make Oracle_development_module to compare two vector field
dev = Oracle_development_module()
# Load development flow
dev.load_differentiation_reference_data(gradient_object=gradient)
# Load simulation result
dev.load_perturb_simulation_data(oracle_object=oracle)
# Calculate inner produc scores
dev.calculate_inner_product()
dev.calculate_digitized_ip(n_bins=10)

In [ ]:
# Show perturbation scores
vm = 0.2
fig, ax = plt.subplots(1, 2, figsize=[12, 5])
dev.plot_inner_product_on_grid(vm=0.02, s=50, ax=ax[0])
ax[0].set_title(f"PS calculated with FCGR2B KO")
dev.plot_inner_product_random_on_grid(vm=vm, s=50, ax=ax[1])
ax[1].set_title(f"PS calculated with Randomized simulation vector")
plt.show()

In [ ]:
# Show perturbation scores with perturbation simulation vector field
fig, ax = plt.subplots(figsize=[6, 6])
dev.plot_inner_product_on_grid(vm=0.05, s=50, ax=ax)
dev.plot_simulation_flow_on_grid(scale=scale_simulation, show_background=False, ax=ax)

In [ ]:
# summary plots
# visualize the results
dev.visualize_development_module_layout_0(s=5, scale_for_simulation=scale_simulation, s_grid=50, scale_for_pseudotime=scale_dev, vm=0.05)